# Simultaneous global and target analysis of the timeresolved fluorescence of PBS at RT and 77K

### Defining datasets and inspect data

In [ ]:
from __future__ import annotations

from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.inspect import show_a_matrixes
from pyglotaran_extras import (
    plot_overview,
    plot_data_overview,
    plot_fitted_traces,
    select_plot_wavelengths,
)

The code below defined the (groups of) datasets used in the analysis. Only for a single dataset the plot_data_overview is shown to avoid repetition, it is left as an execirse to the reader to inspect the other data.

In [ ]:
# order a exc 610 nm and b exc 410 nm
DGDATA_PATH1 = "data/DG_whole_cells_dark_adapted_targeta.ascii"
DGDATA_PATH2 = "data/DG_whole_cells_dark_adapted_targetb.ascii"
DGDATA_PATH3 = "data/DG_whole_cells_after_red_light_targeta.ascii"
DGDATA_PATH4 = "data/DG_whole_cells_after_red_light_targetb.ascii"
DATA_PATH1 = "data/WT_whole_cells_dark_adapted_targeta.ascii"
DATA_PATH2 = "data/WT_whole_cells_dark_adapted_targetb.ascii"
DATA_PATH3 = "data/WT_whole_cells_after_red_light_targeta.ascii"
DATA_PATH4 = "data/WT_whole_cells_after_red_light_targetb.ascii"

plot_data_overview(
    DATA_PATH3,
    nr_of_data_svd_vectors=4,
    linlog=True,
    linthresh=1000,
    irf_location=470,
    title="DA at 77K exc 610 nm",
);

In [ ]:
DATA_PATH650 = "data/guide/20240730DG_WT_PBS_77Kstreak_target_PC650e.ascii"
DATA_PATH660 = "data/guide/20240730DG_WT_PBS_77Kstreak_target_APC660e.ascii"
DATA_PATH680 = "data/guide/20240730DG_WT_PBS_77Kstreak_target_APC680.ascii"
DATA_PATHPSII1= "data/guide/20241111PSI__streak_targetWT_APC680shift4nm.ascii"
DATA_PATHPSII2= "data/guide/20241113streak_WT_target_DA_FRL_PSII2.ascii"
DATA_PATHPSI3= "data/guide/20240720PSend_77Kstreak_target_PSIend.ascii"
DATA_PATHPSII3= "data/guide/20240720PSend_77Kstreak_target_PSIIend.ascii"
plot_data_overview(
    DATA_PATHPSII2,
    nr_of_data_svd_vectors=4,
    linlog=True,
    linthresh=1000,
    irf_location=470,
    title="APC660"
);


# Step 1 Create scheme and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/20250415streak_DG_WT_target_DA_FRL_PSIIdifferent.yml",  # type: ignore
    parameters="models/20250415streak_DG_WT_target_DA_FRL_PSIIdifferent.csv",
    maximum_number_function_evaluations=1,
    clp_link_tolerance=2.1,
    data={
        "DA610": DATA_PATH1,
        "DA410": DATA_PATH2,
        "FRL610": DATA_PATH3,
        "FRL410": DATA_PATH4,
        "data650": DATA_PATH650,
        "data660": DATA_PATH660,
        "data680": DATA_PATH680,
        "dataPSII1": DATA_PATHPSII1,
        "dataPSII2": DATA_PATHPSII2,
        "dataPSII3": DATA_PATHPSII3,
        "DA610DG": DGDATA_PATH1,
        "DA410DG": DGDATA_PATH2,
        "FRL610DG": DGDATA_PATH3,
        "FRL410DG": DGDATA_PATH4,
        "data650DG": DATA_PATH650,
        "data660DG": DATA_PATH660,
        "data680DG": DATA_PATH680,
        "dataPSII1DG": DATA_PATHPSII1,
        "dataPSII2DG": DATA_PATHPSII2,
        "dataPSII3DG": DATA_PATHPSII3,
        "dataPSI3": DATA_PATHPSI3,
        "dataPSI3DG": DATA_PATHPSI3,
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result1 = optimize(target_scheme, raise_exception=True)

For reference, the final Cost should be
-  1         5.1245e+05


In [ ]:
target_result1

In [ ]:
# save_result(
#     result=target_result1,
#     result_path="DGtarget_resultsDA_FRL/result.yaml",
#     allow_overwrite=True,
# )

In [ ]:
target_result1.data["DA610"].lifetime_slowcomplexDA610

# Fig.7 Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas,plot_norm_sas
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.style import ColorCode
from pyglotaran_extras.plotting.style import PlotStyle
myFRLcolors = [
    "r",
    "r",
    "r",
    "k",
    "b",
    "b",
    "b",
    ColorCode.green,
    "g",
    ColorCode.turquoise,
    "tab:orange",
    ColorCode.magenta,
    ColorCode.indigo,
]
res=target_result1.data["FRL610"]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 13)
custom_cycler3 = cycler(color=myFRLcolors, linestyle=[":"] * 13)
custom_cycler4 = cycler(color=myFRLcolors, linestyle=["-."] * 13)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(target_result1.data["FRL610"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler3,linetype=":")
plot_concentrations(target_result1.data["DA610"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler)
plot_sas(target_result1.data["DA610"], axes[1], cycler=custom_cycler)
plot_norm_sas(target_result1.data["DA610"], axes[2], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_xlim(-1500,5500)
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

# Fig.S21 A-C result WT

In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas,plot_norm_sas
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.style import ColorCode
from pyglotaran_extras.plotting.style import PlotStyle
res=target_result1.data["FRL610"]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 13)
custom_cycler3 = cycler(color=myFRLcolors, linestyle=[":"] * 13)
custom_cycler4 = cycler(color=myFRLcolors, linestyle=["-."] * 13)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(target_result1.data["DA410"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler2,linetype="--")
plot_concentrations(target_result1.data["FRL610"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler3,linetype=":")
plot_concentrations(target_result1.data["FRL410"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler4,linetype="-.")
plot_concentrations(target_result1.data["DA610"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler)
plot_sas(target_result1.data["DA610"], axes[1], cycler=custom_cycler)
plot_sas(target_result1.data["DA410"], axes[1], cycler=custom_cycler2)
plot_sas(target_result1.data["FRL610"], axes[1], cycler=custom_cycler3)
plot_sas(target_result1.data["FRL410"], axes[1], cycler=custom_cycler4)
plot_norm_sas(target_result1.data["DA610"], axes[2], cycler=custom_cycler)
plot_norm_sas(target_result1.data["DA410"], axes[2], cycler=custom_cycler2)
plot_norm_sas(target_result1.data["FRL610"], axes[2], cycler=custom_cycler3)
plot_norm_sas(target_result1.data["FRL410"], axes[2], cycler=custom_cycler4)
axes[0].set_xlabel("Time (ps)")
axes[0].set_xlim(-1500,5500)
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

# Fig.S21 D-F result DG 

In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas,plot_norm_sas
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.style import ColorCode
from pyglotaran_extras.plotting.style import PlotStyle
res=target_result1.data["FRL610"]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 13)
custom_cycler3 = cycler(color=myFRLcolors, linestyle=[":"] * 13)
custom_cycler4 = cycler(color=myFRLcolors, linestyle=["-."] * 13)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(target_result1.data["DA410DG"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler2,linetype="--")
plot_concentrations(target_result1.data["FRL610DG"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler3,linetype=":")
plot_concentrations(target_result1.data["FRL410DG"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler4,linetype="-.")
plot_concentrations(target_result1.data["DA610DG"], axes[0], center_λ=0, linlog=True, linthresh=1000, cycler=custom_cycler)
plot_sas(target_result1.data["DA610DG"], axes[1], cycler=custom_cycler)
plot_sas(target_result1.data["DA410DG"], axes[1], cycler=custom_cycler2)
plot_sas(target_result1.data["FRL610DG"], axes[1], cycler=custom_cycler3)
plot_sas(target_result1.data["FRL410DG"], axes[1], cycler=custom_cycler4)
plot_norm_sas(target_result1.data["DA610DG"], axes[2], cycler=custom_cycler)
plot_norm_sas(target_result1.data["DA410DG"], axes[2], cycler=custom_cycler2)
plot_norm_sas(target_result1.data["FRL610DG"], axes[2], cycler=custom_cycler3)
plot_norm_sas(target_result1.data["FRL410DG"], axes[2], cycler=custom_cycler4)
axes[0].set_xlabel("Time (ps)")
axes[0].set_xlim(-1500,5500)
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("D", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("E", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("F", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

# Guide fits


In [ ]:
fig, axes = plt.subplots(2, 6, figsize=(15, 7))
target_result1.data["data650"].data.plot(ax=axes[0,0])
target_result1.data["data650"].fitted_data.plot(ax=axes[0,0])
target_result1.data["data660"].data.plot(ax=axes[0,1])
target_result1.data["data660"].fitted_data.plot(ax=axes[0,1])
target_result1.data["data680"].data.plot(ax=axes[0,2])
target_result1.data["data680"].fitted_data.plot(ax=axes[0,2])
target_result1.data["dataPSII1"].data.plot(ax=axes[0,3])
target_result1.data["dataPSII1"].fitted_data.plot(ax=axes[0,3])
target_result1.data["dataPSII2"].data.plot(ax=axes[0,4])
target_result1.data["dataPSII2"].fitted_data.plot(ax=axes[0,4])
target_result1.data["dataPSII3"].data.plot(ax=axes[0,5])
target_result1.data["dataPSII3"].fitted_data.plot(ax=axes[0,5])
axes[0,0].set_xlabel("")
axes[0,0].set_ylabel("SAS")
axes[0,0].set_title("PC650")
axes[0,1].set_xlabel("")
axes[0,1].set_ylabel("SAS")
axes[0,1].set_title("APC660")
axes[0,2].set_xlabel("")
axes[0,2].set_ylabel("")
axes[0,2].set_title("APC680")
axes[0,3].set_xlabel("")
axes[0,3].set_ylabel("")
axes[0,3].set_title("PSII1")
axes[0,4].set_xlabel("")
axes[0,4].set_ylabel("")
axes[0,4].set_title("PSII2")
axes[0,5].set_ylabel("")
axes[0,5].set_title("PSII3")
target_result1.data["data650DG"].data.plot(ax=axes[1,0])
target_result1.data["data650DG"].fitted_data.plot(ax=axes[1,0])
target_result1.data["data660DG"].data.plot(ax=axes[1,1])
target_result1.data["data660DG"].fitted_data.plot(ax=axes[1,1])
target_result1.data["data680DG"].data.plot(ax=axes[1,2])
target_result1.data["data680DG"].fitted_data.plot(ax=axes[1,2])
target_result1.data["dataPSII1DG"].data.plot(ax=axes[1,3])
target_result1.data["dataPSII1DG"].fitted_data.plot(ax=axes[1,3])
target_result1.data["dataPSII2DG"].data.plot(ax=axes[1,4])
target_result1.data["dataPSII2DG"].fitted_data.plot(ax=axes[1,4])
target_result1.data["dataPSII3DG"].data.plot(ax=axes[1,5])
target_result1.data["dataPSII3DG"].fitted_data.plot(ax=axes[1,5])
axes[1,0].set_xlabel("Wavelength (nm)")
axes[1,0].set_ylabel("SAS")
axes[1,0].set_title("PC650")
axes[1,1].set_xlabel("Wavelength (nm)")
axes[1,1].set_ylabel("SAS")
axes[1,1].set_title("APC660")
axes[1,2].set_xlabel("Wavelength (nm)")
axes[1,3].set_xlabel("Wavelength (nm)")
axes[1,4].set_xlabel("Wavelength (nm)")
axes[1,5].set_xlabel("Wavelength (nm)")
axes[1,2].set_ylabel("")
axes[1,2].set_title("APC680")
axes[1,3].set_ylabel("")
axes[1,3].set_title("PSII1")
axes[1,4].set_ylabel("")
axes[1,4].set_title("PSII2")
axes[1,5].set_ylabel("")
axes[1,5].set_title("PSII3")

# overview FRL610

In [ ]:
das_cycler = cycler(color=PlotStyle().cycler)
fig, axes = plot_overview(target_result1.data["FRL610"], linlog=True, linthresh=1000,cycler=custom_cycler,das_cycler=das_cycler,nr_of_residual_svd_vectors=1,svd_cycler=das_cycler)


In [ ]:
from custom_plotting import plot_residual_and_svd
from custom_plotting import plot_svd_of_residual


### Residual analysis of all data


        "DA610": DATA_PATH1,
        "DA410": DATA_PATH2,
        "FRL610": DATA_PATH3,
        "FRL410": DATA_PATH4,
        "DA610DG": DGDATA_PATH1,
        "DA410DG": DGDATA_PATH2,
        "FRL610DG": DGDATA_PATH3,
        "FRL410DG": DGDATA_PATH4,


In [ ]:
fig, axes = plot_svd_of_residual(
    [
        target_result1.data["DA610"],  # order! grey, k, orange, r
        target_result1.data["DA410"],
        target_result1.data["FRL610"],
        target_result1.data["FRL410"],
    ],
    linlog=True,
    linthresh=1000,
    index=0,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

In [ ]:
fig, axes = plot_svd_of_residual(
    [
        target_result1.data["DA610DG"],  # order! grey, k, orange, r
        target_result1.data["DA410DG"],
        target_result1.data["FRL610DG"],
        target_result1.data["FRL410DG"],
    ],
    linlog=True,
    linthresh=1000,
    index=0,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

# Print the estimates of the optimized parameters and their precision 
The t-values of the **free** parameters (Vary=True) indicate the precision

In [ ]:
target_result1.optimized_parameters

## Fit quality of the target analysis of the WL data
overlays of traces and fits, first of 16 wavelengths, then of 6 selected wavelengths

# Fig.S17 traces with fits WT

In [ ]:
target_result_streak = (
    target_result1.data["FRL410"],
    target_result1.data["FRL610"],
    target_result1.data["DA410"],
    target_result1.data["DA610"],
)

In [ ]:
import numpy as np
wavelengths =np.linspace(665, 740, num=16)
wavelengths =np.linspace(645, 740, num=20)
wavelengths =(666,690,700, 725)
fig, ax_= plot_fitted_traces(target_result_streak, wavelengths, axes_shape=(2,2), figsize=(15,10),linlog=True, per_axis_legend=True,linthresh=1000);
handles, labels = ax_.flatten()[0].get_legend_handles_labels()
for i in range(len(handles)):
    if i == 1:
        labels[i] = "FRL410"
    elif i == 3:
        labels[i] = "FRL610"
    elif i == 5:
        labels[i] = "DA410"
    elif i == 7:
        labels[i] = "DA610"
    else:
        labels[i] = "_Hidden"
for idx, ax in enumerate(ax_.flatten()):
    ax_title = ax.get_title()
    ax.set_title(rf"{ax_title.replace('spectral', 'Wavelength')}$\,$ nm")
    ax.set_ylabel("")
    ax.set_yticklabels([])
    if idx > 1:
        ax.set_xlabel("Time (ps)")
    else:
        ax.set_xlabel("")
    if ax.get_legend() is not None:
        ax.get_legend().remove()
    for line in ax.lines:
        line.set_linewidth(1)  # Set the line width here
labels[1]='State I 410 nm exc'
labels[3]='State I 610 nm exc'
labels[5]='State II 410 nm exc'
labels[7]='State II 610 nm exc'
fig.legend(
    handles,
    labels,
    bbox_to_anchor=(0.5, -0.05),
    loc="lower center",
    ncol=len(handles),fontsize=18,
)
fig.tight_layout()

In [ ]:
labels

In [ ]:
import numpy as np
wavelengths =np.linspace(665, 740, num=16)
wavelengths =np.linspace(645, 740, num=20)
fig, ax_= plot_fitted_traces(target_result_streak, wavelengths, axes_shape=(4,5), linlog=True, per_axis_legend=True,linthresh=1000);
handles, labels = ax_.flatten()[0].get_legend_handles_labels()
for i in range(len(handles)):
    if i == 1:
        labels[i] = "FRL410"
    elif i == 3:
        labels[i] = "FRL610"
    elif i == 5:
        labels[i] = "DA410"
    elif i == 7:
        labels[i] = "DA610"
    else:
        labels[i] = "_Hidden"
for idx, ax in enumerate(ax_.flatten()):
    ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
    if idx > 14:
        ax.set_xlabel("Time (ps)")
    else:
        ax.set_xlabel("")
    ax.set_title("")
    if ax.get_legend() is not None:
        ax.get_legend().remove()
    for line in ax.lines:
        line.set_linewidth(1)  # Set the line width here
fig.legend(
    handles,
    labels,
    bbox_to_anchor=(0.5, -0.05),
    loc="lower center",
    ncol=len(handles),fontsize=24,
)
fig.tight_layout()

# Fig.S18 traces with fits DG

In [ ]:
target_result_streakDG = (
    target_result1.data["FRL410DG"],
    target_result1.data["FRL610DG"],
    target_result1.data["DA410DG"],
    target_result1.data["DA610DG"],
)

In [ ]:
import numpy as np
wavelengths =np.linspace(665, 740, num=16)
wavelengths =np.linspace(645, 740, num=20)
fig, ax_= plot_fitted_traces(target_result_streakDG, wavelengths, axes_shape=(4,5), linlog=True, per_axis_legend=True,linthresh=1000);
handles, labels = ax_.flatten()[0].get_legend_handles_labels()
for i in range(len(handles)):
    if i == 1:
        labels[i] = "FRL410"
    elif i == 3:
        labels[i] = "FRL610"
    elif i == 5:
        labels[i] = "DA410"
    elif i == 7:
        labels[i] = "DA610"
    else:
        labels[i] = "_Hidden"
for idx, ax in enumerate(ax_.flatten()):
    ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
    if idx > 14:
        ax.set_xlabel("Time (ps)")
    else:
        ax.set_xlabel("")
    ax.set_title("")
    if ax.get_legend() is not None:
        ax.get_legend().remove()
    for line in ax.lines:
        line.set_linewidth(1)  # Set the line width here
fig.legend(
    handles,
    labels,
    bbox_to_anchor=(0.5, -0.05),
    loc="lower center",
    ncol=len(handles),fontsize=24,
)
fig.tight_layout()

# overview FRL610

## Compute the FWHM of the main Gaussian of the IRF

In [ ]:
import numpy as np
const=2*np.sqrt(2*np.log(2))
[const*target_result1.optimized_parameters.get("DA410irf.width").value,
 const*target_result1.optimized_parameters.get("FRL410irf.width").value,
 const*target_result1.optimized_parameters.get("DA610irf.width").value,
 const*target_result1.optimized_parameters.get("FRL610irf.width").value]

In [ ]:
import numpy as np
const=2*np.sqrt(2*np.log(2))
[const*target_result1.optimized_parameters.get("DA410DGirf.width").value,
 const*target_result1.optimized_parameters.get("FRL410DGirf.width").value,
 const*target_result1.optimized_parameters.get("DA610DGirf.width").value,
 const*target_result1.optimized_parameters.get("FRL610DGirf.width").value]

In [ ]:
fig, axes = plot_overview(target_result1.data["FRL610"], linlog=True, linthresh=1000,cycler=custom_cycler)


In [ ]:
fig, axes = plot_overview(target_result1.data["FRL410"], linlog=True, linthresh=1000,cycler=custom_cycler)


In [ ]:
fig, axes = plot_overview(target_result1.data["DA610"], linlog=True, linthresh=1000,cycler=custom_cycler)


In [ ]:
fig, axes = plot_overview(target_result1.data["DA410"], linlog=True, linthresh=1000,cycler=custom_cycler)
